In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

from collections import defaultdict, Counter
from tqdm import tqdm_notebook

%matplotlib inline

In [4]:
class OnlineLogisticRegression():
    
    """ OnlineLogisticRegression classifier
    Класс реализует модель multiclass/multilabel классификации текстов, используя методы онлайн 
    обучения (стохастический градиентный спуск). В качестве регуляризатора используется ElasticNet 
    (комбинация L1 и L2). Поддерживаются повторные проходы по тренировочному датасету и ограничения 
    на размер словаря.
    
    Parameters
    ----------
    tags : [string]
        Список допустимых для классификации тегов (классов). Не входящие в этот список теги 
        игнорируются.

    strategy : ['ovr', 'multinomial'], default: 'ovr'
        Признак типа классификации. Значение 'ovr' задает бинарную классификацию (присутствие/отсутствие) 
        для каждого тега, теги независимы (one-vs-rest, multilabel classification). Значение 
        'multinomial' задает минимизацию ошибки общего дискретного вероятностного распределения 
        (multiclass classification).
    
    learning_rate : float, default: 0.1
        Скорость обучения градиентного спуска (множитель корректировки параметра модели на каждом шаге).

    lmbda : float, default: 0.0002
        Коэффициент ElasticNet-регуляризации на каждом шаге.

    gamma : float, default: 0.1
        Вес L2-компоненты в ElasticNet.

    tolerance : float, default: 1e-16
        Порог для огнаничения значений аргумента логарифма.
    
    Attributes
    ----------
    vocab_ : dict {string: int}
        Mapping слов-признаков в численные индексы. Слова добавляются в словарь в процессе обучения, 
        индексы назначаются инкрементально.

    w_ : dict {string: defaultdict(int)}
        Mapping тегов в словарь {<численный_индекс_признака>: <вес_в_модели>} (для каждого тега
        свой набор параметров модели). Словарь изменяемого размера со значением по умолчанию 0.
        
    w0_ : dict {string: float}
        Mapping тегов в веса w0 (смещения).
        
    train_frequency_dict_ : Counter
        Counter-объект {<численный_индекс_признака>: <число_вхождений>}. Выполняет подсчет числа
        вхождений признака на всей тренировочной выборке.
    
    loss_ : [double]
        Список значений функции потерь для последней используемой обучающей выборки.
    
    """
    def __init__(self, tags, strategy='ovr', learning_rate=0.1, lmbda=0.0002, gamma=0.1, tolerance=1e-16):
        self.vocab_ = {}
        self._w = {t: defaultdict(int) for t in tags}
        self._b = {t: 0.0 for t in tags}
        self._train_frequency_dict = Counter()
        
        self.tags_ = set(tags)
        self.strategy_ = strategy
        self.learning_rate_ = learning_rate
        self.lmbda_ = lmbda
        self.gamma_ = gamma
        self.tolerance_ = tolerance
        
    
    def fit(self, datasource, total=None, update_vocab=True, return_train_loss=False):
        """Fit/update the model by passing the datasource
        Обучение/дообучение модели одним проходом по источнику данных.

        Parameters
        ----------
        datasource : iterable
            Итерируемый объект как источник данных. Формат примера из обучающей выборки - строка
            с классифицируемым текстом и список тегов, разделенные знаком табуляции.

        total : int or None, default=None
            Информация о количестве строк в источнике данных для вывода прогресс-бара. В случае
            значения None, прогресс-бар не используется.

        update_vocab : bool, default=True
            Флаг режима добавления слов в словарь (признаковое пространство) во время обучения.
            
        return_train_loss : bool, default=False
            Флаг сохранения значений функции потерь для каждого примера из обучающей выборки.
            
        Returns
        -------
        self : object
            Возвращает объект класса
        """
        self.loss_ = [] 
        
        if total is not None:
            wrapped_source = tqdm_notebook(datasource, total=total, mininterval=1)
        else:
            wrapped_source = datasource
            
        for line in wrapped_source:
            input_pair = line.strip().split('\t')
            if len(input_pair) != 2:
                continue             
                
            word_sentence, tags = input_pair
            word_sentence = word_sentence.split(' ')
            tags = set(tags.split(' '))

            # значение функции потерь для текущего примера
            sample_loss = 0

            # прокидываем градиенты для каждого тега
            for tag in self._tags:
                # целевая переменная равна 1 если текущий тег есть у текущего примера
                y = int(tag in tags)

                # расчитываем значение линейной комбинации весов и признаков объекта
                # инициализируем z
                z = self._b[tag]

                for word in sentence:

                    if word not in self._vocab:
                        # если в режиме тестирования появляется слово которого нет в словаре, то мы его игнорируем
                        if n >= top_n_train:
                            continue
                        # если встречаем новое слово с запретом на добавление, игнорируем его
                        if not update_vocab:
                            continue

                        self._vocab[word] = len(self._vocab)

                    z += self._w[tag][self._vocab[word]] * 1.0

                # вычисляем вероятность наличия тега
                if z >= 0:
                    sigma = 1 / (1 + np.exp(-z))
                else:
                    sigma = 1 - 1 / (1 + np.exp(z))

                # обновляем значение функции потерь для текущего примера
                if y == 1:
                    sample_loss += -1 * np.log(np.max([sigma, tolerance]))
                else:
                    sample_loss += -1 * np.log(1 - np.min([1 - tolerance, sigma]))

                # если мы все еще в тренировочной части, то обновим параметры
                if n < top_n_train:
                    # вычисляем производную логарифмического правдоподобия по весу

                    # учет xm будет реализовываться в цикле далее
                    dLdw = (y - sigma)

                    # делаем градиентный шаг
                    # мы минимизируем отрицательное логарифмическое правдоподобие (второй знак минус)
                    # поэтому мы идем в обратную сторону градиента для минимизации (первый знак минус)
                    regularized_words = set()

                    for word in sentence: 
                        # игнорируем слово не из словаря
                        if word not in self._vocab:
                            continue

                        # регуляризация веса только для первого вхождения слова
                        if(self._vocab[word] not in regularized_words):
                            w = self._w[tag][self._vocab[word]]
                            regularization = lmbda * (2 * gamma * w + (1 - gamma) * np.sign(w))
                            self._w[tag][self._vocab[word]] -= learning_rate * regularization
                            regularized_words.add(self._vocab[word])

                        # корректировка веса
                        self._w[tag][self._vocab[word]] -= -learning_rate*dLdw

                    self._b[tag] -= -learning_rate*dLdw

                # в тестовой части посчитаем прогноз    
                else:
                    if sigma > predict_threshold:
                        predicted_tags.append(tag)

            self._loss.append(sample_loss)

            # обновим частотный словарь
            if n < top_n_train:
                if update_vocab:
                    self._train_frequency_dict += Counter([self._vocab[word] for word in sentence])

            # метрика качества для прогноза
            else:
                k_jaccard = len(set(predicted_tags) & set(tags)) / len(set(predicted_tags) | set(tags))
                self._accuracy.append(k_jaccard)
                #print('tags: {} predicted: {} k_jaccard={}'.format(tags, predicted_tags, k_jaccard))
                
        return self

    
    """Отбор топ-n самых популярных слов в словаре (для всех тегов)
    
    Параметры
    ----------
    n : int, default=10000
        количество слов для отбора
    """
    def filter_vocab(self, n=10000):
        
        # мнлжество топ-n популярных слов
        top_common_w = {k for (k, v) in self._train_frequency_dict.most_common(n)}

        # обновим словарь
        self._vocab = {key: val for (key, val) in self._vocab.items() if val in top_common_w}
        
        # обновим словари для тегов
        for tag in self._tags:
            self._w[tag] = {key: val for (key, val) in self._w[tag].items() if key in top_common_w}
            
    """Предсказание тегов для вопроса
    
    Параметры
    ----------
    sentence : string
        текст вопроса
    """
    def predict_proba(self, sentence):        
        
        sentence = sentence.strip().split(' ')
        predicted = dict()
        
        for tag in self._tags:
            # расчитываем значение линейной комбинации весов и признаков объекта
            z = self._b[tag]
            
            for word in sentence:
                if word not in self._vocab:
                    continue
                z += self._w[tag][self._vocab[word]] * 1.0

            # вычисляем вероятность наличия тега
            if z >= 0:
                sigma = 1 / (1 + np.exp(-z))
            else:
                sigma = 1 - 1 / (1 + np.exp(z))

            predicted[tag] = sigma
        
        return predicted